**GİRİŞ**

* Bu projemizde Turkish cyberbullying datasını kullanarak; Türkçe hakaret içeren cümleleri tespit edebilen bir model eğiteceğiz. Bu model içinde CountVectorizer ve TFIDF Vectorizer'ları ile tanışıp, Pipeline yapısını kullanmayı öğreneceğiz. En son olarak da modelimizi Joblib ile '.pkl' uzantılı bir dosyaya kaydedip, daha sonraki kullanımlarımızda yalnızca bu dosyayı çağırarak işlem yapacağız.

* Benim ilk NLP ve ilk Türkçe kernel'ım olacak bu çalışma. Dolayısıyla çok heyecalıyım. O zaman başlayalım!

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['turkish cyberbullying.csv']


In [2]:
data = pd.read_csv('../input/turkish cyberbullying.csv')

In [3]:
data.head()

,message,cyberbullying
0,rabbim kalan ömrünü geçen ömründen hayırlı eyl...,0
1,bir ateist olarak bu resmi gördükçe gözyaşları...,0
2,oo süpersin azıcık bize de bulaşsa,0
3,bende biliyorum benden bı bok olmicak,1
4,nerdesin len tirrek,1


In [4]:
data.tail(10)

,message,cyberbullying
2991,çikolata kakaodan yapılıyor kakao ise ağacın t...,0
2992,sen çok kafana takıyorsun takıyorum kardeşim k...,1
2993,birinden kaçtin ama diğerinden kaçamadınbu ülk...,0
2994,düğün var diye mutlu olan var mı allah aşkına ...,0
2995,vefa ile ne alaka vefanın anlamını biliyor mu ...,0
2996,bismillahirahmanirahim yaallah hayırlı sabahla...,0
2997,jskdpdğeff deli amk,1
2998,nasıl bir yavşak serefsizsiniz hepiniz,1
2999,shipshipshiphayırlı olsun,0
3000,çookk tatlısın çok,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3001 entries, 0 to 3000
Data columns (total 2 columns):
message          3001 non-null object
cyberbullying    3001 non-null int64
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [6]:
data.describe()

,cyberbullying
count,3001.000000
mean,0.500833
std,0.500083
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [7]:
y = data.cyberbullying.values
x = data.message.values

In [8]:
print(x)
print(len(x))

['rabbim kalan ömrünü geçen ömründen hayırlı eylesin'
 'bir ateist olarak bu resmi gördükçe gözyaşlarıma mani olamıyorum'
 'oo süpersin azıcık bize de bulaşsa' ...
 'nasıl bir yavşak serefsizsiniz hepiniz' 'shipshipshiphayırlı olsun'
 'çookk tatlısın çok']
3001


In [9]:
x.shape
y.shape

(3001,)

Train - Test Split

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,random_state=42)

In [11]:
print(len(x_train))
print(len(x_test))

2250
751


In [12]:
veri = x.copy()

In [13]:
veri.shape

(3001,)

Türkçe'de bulunan '**Stopwords**'(gereksiz kelimeler)'den kurtulmamız gerekiyor çünkü bu kelimeler classification yaparken işimize yaramayacak kelimeler. O nedenle bu kelimelerden kurtulalım!

In [14]:
from nltk.corpus import stopwords
stop = stopwords.words("turkish")

**CountVectorizer**

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=5,stop_words=stop,ngram_range=(1,3))
vectorizer.fit(veri)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 3), preprocessor=None,
        stop_words=['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [16]:
BoW = vectorizer.transform(veri)
repr(BoW)

"<3001x795 sparse matrix of type '<class 'numpy.int64'>'\n\twith 11001 stored elements in Compressed Sparse Row format>"

In [17]:
feature_names = vectorizer.get_feature_names()
print("100 ile 110 arasındaki değerler:\n{}".format(feature_names[100:110]))

100 ile 110 arasındaki değerler:
['ben', 'bence', 'bende', 'benden', 'beni', 'benim', 'beraber', 'berduş', 'beri', 'bey']


**TFIDF Vectorizer**

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
for min_df in [1,2,3,4,5,6]:
    for n_gram in [(1,1),(1,2),(1,3),(2,3)]:
        tf_vectorizer = TfidfVectorizer(min_df=min_df, stop_words=stop,ngram_range=n_gram)
        veri1 = tf_vectorizer.fit_transform(veri)
        best = veri1.max(axis=0).toarray().ravel()
        sort_by_tfidf = best.argsort()
        feature_names = np.array(tf_vectorizer.get_feature_names())
        print("Vocabularies using min_df={} and n_gram={} with highest tfidf: \n{}".format(min_df, n_gram, feature_names[sort_by_tfidf[-20:]]))
        print("The number of vocabularies: {}".format(len(tf_vectorizer.vocabulary_)))
        sort_by_tfidf = np.argsort(tf_vectorizer.idf_)
        print("Vocabularies with lowest idf:\n{}".format(feature_names[sort_by_tfidf[:20]]))
        print('-----------------------------------')

Vocabularies using min_df=1 and n_gram=(1, 1) with highest tfidf: 
['sınav' 'uzat' 'gittin' 'sıffır' 'hödük' 'vay' 'tarzdır' 'hee' 'yalayın'
 'styling' 'güzelsin' 'tatlısın' 'tırrek' 'andaval' 'sisko' 'şişko'
 '09052018' 'diyarbakır' 'tatlı' 'yobaz']
The number of vocabularies: 11458
Vocabularies with lowest idf:
['bir' 'sen' 'kadar' 'ben' 'var' 'bi' 'senin' 'orospu' 'amk' 'seni'
 'olsun' 'lan' 'gerizeka' 'mutluyum' 'yok' 'pislik' 'mi' 'güzel' 'andaval'
 'değil']
-----------------------------------
Vocabularies using min_df=1 and n_gram=(1, 2) with highest tfidf: 
['güzelsin kız' 'güzelsin çocuk' 'herkes gerizeka' 'uzat' 'güzelsin sen'
 'güzelsin güzel' 'ulan amk' 'hee' 'yalayın' 'diyarbakır' 'andaval'
 'sisko' 'tırrek' 'tatlısın' '09052018' 'styling' 'yobaz' 'tatlı' 'şişko'
 'güzelsin']
The number of vocabularies: 32321
Vocabularies with lowest idf:
['bir' 'sen' 'ben' 'kadar' 'var' 'bi' 'senin' 'orospu' 'amk' 'seni'
 'olsun' 'lan' 'gerizeka' 'mutluyum' 'yok' 'pislik' 'mi' 'güzel' 'and

Modelimizi eğitirken kullanacağımız metotlarımızı import ediyoruz.

In [19]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score

Çalışmamda Pipeline yapısı kullanacağım. Pipeline'ı bilmeyenler için => https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

In [21]:
from sklearn.pipeline import Pipeline

Modelde kullanacağımız metotları, vectorizerları da kullanarak tek tek tanıtmaya başlıyorum.

In [22]:

LinearSVC_count = Pipeline([
    ('countvectorizer',CountVectorizer()),
    ('LinearSVC',LinearSVC(max_iter=1000))
])

LinearSVC_tfidf = Pipeline([
        ('tfidfvectorizer', TfidfVectorizer()),
        ('LinearSVC', LinearSVC(max_iter=1000))
])

In [23]:
Naive_count = Pipeline([
        ('countvectorizer', CountVectorizer()),
        ('multinomialnb', MultinomialNB())
])

Naive_tfidf = Pipeline([
        ('tfidfvectorizer', TfidfVectorizer()),
        ('multinomialnb', MultinomialNB())
])

In [24]:
Decision_count = Pipeline([
        ('countvectorizer', CountVectorizer()),
        ('decisiontreeclassifier', DecisionTreeClassifier())
])

Decision_tfidf = Pipeline([
        ('tfidfvectorizer', TfidfVectorizer()),
        ('decisiontreeclassifier', DecisionTreeClassifier())
])

In [25]:
RandomForest_count = Pipeline([
        ('countvectorizer', CountVectorizer()),
        ('randomforestclassifier', RandomForestClassifier(n_estimators=100))
])

RandomForest_tfidf = Pipeline([
        ('tfidfvectorizer', TfidfVectorizer()),
        ('randomforestclassifier', RandomForestClassifier(n_estimators=100))
])

Şimdi parametrelerimizi tanıtalım. Metotların içinden yalnızca LinearSVC hyper parameter olarak 'C' parametresini alıyor. O nedenle LinearSVC parametrelerini ayrı, kalan 3 metotun parametrelerini ayrı alacağım.

In [26]:
#İlk olarak bünyesinde Count Vectorizer olan parametreler.
parameters_of_svc_count = [ 
    {
        'LinearSVC__C': [0.01, 0.1, 1, 10, 100], 
        'countvectorizer__min_df': [1,3,5], 
        'countvectorizer__stop_words': [None, stop],
        'countvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 3)]
    } 
]

parameters_general_count = [ 
    {
        'countvectorizer__min_df': [1,3,5], 
        'countvectorizer__stop_words': [None, stop],
        'countvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 3)]
    }
]

In [27]:
parameters_of_svc_tfidf = [ 
    {
        'LinearSVC__C': [0.01, 0.1, 1, 10, 100], 
        'tfidfvectorizer__min_df': [1,3,5], 
        'tfidfvectorizer__stop_words': [stop],
        'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 3)]
    } 
]

parameters_of_general_tfidf = [ 
    {
        'tfidfvectorizer__min_df': [1,3,5], 
        'tfidfvectorizer__stop_words': [stop],
        'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 3)]
    }
]

In [28]:
x_train.shape

(2250,)

Şimdi modellerimizi çalıştırmaya başlayalım. İlk olarak CountVectorizer içeren modeller.

In [29]:
for models, parameters, name in zip([LinearSVC_count, Naive_count, Decision_count, RandomForest_count],
                                    [parameters_of_svc_count, parameters_general_count, parameters_general_count, parameters_general_count],
                                    ["LinearSVC","Multinomial NB","Decision Tree","Random Forest"]):

    grid = GridSearchCV(models, parameters, cv=5)
    grid.fit(x_train, y_train)
    print("Model ismi: "+ name)
    print("En iyi cross-validation score: {:.2f}".format(grid.best_score_ * 100))
    print("En iyi parametreler: ", grid.best_params_)
    
    y_train_pred = grid.predict(x_train)
    print(confusion_matrix(y_train, y_train_pred))

    
    final_model = grid.best_estimator_
    final_test_prediction = final_model.score(x_test, y_test)
    print("Test score: {:.2f}%".format(final_test_prediction * 100))    
    print("--------------------------")

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Model ismi: LinearSVC
En iyi cross-validation score: 89.24
En iyi parametreler:  {'LinearSVC__C': 1, 'countvectorizer__min_df': 1, 'countvectorizer__ngram_range': (1, 2), 'countvectorizer__stop_words': ['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']}
[[1118    0]
 [   0 1132]]
Test score: 89.21%
--------------------------
Model ismi: Multinomial NB
En iyi cross-validation score: 87.20
En iyi parametreler:  {'countvectorizer__min_df': 1, 'countvectorizer__ngram_range': (1, 1), 'countvectorizer__stop_words': None}
[[1102   16]
 [   3 1129]]
Test score: 86.42%
--------------------------
Model ismi: Decision Tree
En iyi cross-validation score

Şimdi de TFIDF vectorizer'ına göre sonuç alalım.

In [30]:
for models, parameters, name in zip([LinearSVC_tfidf, Naive_tfidf, Decision_tfidf, RandomForest_tfidf],
                                    [parameters_of_svc_tfidf, parameters_of_general_tfidf, parameters_of_general_tfidf, parameters_of_general_tfidf],
                                    ["LinearSVC","Multinomial Naive Bayes","Decision Tree","Random Forest"]):

    grid = GridSearchCV(models, parameters, cv=5)
    grid.fit(x_train, y_train)
    print("Model ismi: "+ name)
    print("En iyi cross-validation score: {:.2f}".format(grid.best_score_ * 100))
    print("En iyi parametreler: ", grid.best_params_)
    
    y_train_pred = grid.predict(x_train)
    print(confusion_matrix(y_train, y_train_pred))
    
    final_model = grid.best_estimator_
    final_test_prediction = final_model.score(x_test, y_test)
    print("Test score: {:.2f}%".format(final_test_prediction * 100))    
    print("--------------------------")

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Model ismi: LinearSVC
En iyi cross-validation score: 88.44
En iyi parametreler:  {'LinearSVC__C': 10, 'tfidfvectorizer__min_df': 1, 'tfidfvectorizer__ngram_range': (1, 2), 'tfidfvectorizer__stop_words': ['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']}
[[1118    0]
 [   0 1132]]
Test score: 90.15%
--------------------------
Model ismi: Multinomial Naive Bayes
En iyi cross-validation score: 86.18
En iyi parametreler:  {'tfidfvectorizer__min_df': 1, 'tfidfvectorizer__ngram_range': (1, 3), 'tfidfvectorizer__stop_words': ['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'd

En iyi test score'u veren modelimiz TFIDF vectorizer'ı ile LinearSVC modeli oldu. En iyi parametreler: C=10, min_df=1, stop_words=word, ngram_range=(1,2). Score: %90.15

En iyi sonuç aldığımız modeli parametreleri ve vectorizer'ıyla birlikte yeni bir pipeline yapısında tutalım. Yeni gelen değerler direk bu yeni yarattığımız yapıya gidecek ve orada kontrol ettirilecek.

En iyi sonuç aldığımız modelimizin pipeline yapısı:

In [31]:
# En iyi sonuç veren parametrelerimizi de yapının içerisine ekliyoruz.
best_pipeline = Pipeline([
    ('tfidfvectorizer', TfidfVectorizer(min_df=1,stop_words=stop,ngram_range=(1,2))),
    ('LinearSVC', LinearSVC(C=10,max_iter=1000))
])

In [32]:
best_pipeline.fit(x_train,y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [33]:
best_pipeline.steps

[('tfidfvectorizer',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani'],
          strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('LinearSVC',
  LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1

In [34]:
final_test_prediction = best_pipeline.score(x_test,y_test)
print("Final test score:",final_test_prediction)

y_test_pred = best_pipeline.predict(x_test)
print("Confusion matrix\n",confusion_matrix(y_test,y_test_pred))

Final test score: 0.9014647137150466
Confusion matrix
 [[335  45]
 [ 29 342]]


Bu modelimizi bir dosya içerisine save etmemiz gerekiyor şu an. Dosyanın adını en 'latest_model' yapacağım.

In [35]:
from sklearn.externals import joblib

joblib.dump(best_pipeline,"latest_model.pkl")

['latest_model.pkl']

Dosyayı load ederek deneyelim.

In [36]:
best_model = joblib.load("latest_model.pkl")
final_test_prediction = best_model.score(x_test,y_test)
print("Final Test Score:",final_test_prediction)

Final Test Score: 0.9014647137150466


Gördüğümüz gibi; dosyamızın içinde bulunan final score'u önümüze geldi!

Şimdi kendi yazdığımız bir kaç örnekle classification yapabiliyor muyuz test edelim!

In [37]:
best_model.predict(['gayet iyi','salak','bu ne amk','güzelsin','siktir','hava çok güzel'])

array([0, 1, 1, 0, 1, 0])

Gördüğünüz gibi hakaret içeren ifadeler 1 nolu (hakaret içeren ifadeler) hakaret içermeyen ifadeler 0 nolu (hakaret edilmeyen) class'ımıza classify yapılmış oldu. Modelimiz gayet sağlıklı çalışıyor.

**SONUÇ**
* Modelimizi en iyi şekilde eğittik ve bir cümle içerisinde hakaret olup olmadığını bulan bir model geliştirdik. Son bölümde yaptığımız testlerde de açıkça görüldüğü gibi; modelimiz gayet güzel şekilde çalışıyor. Predict ederken kullandığım kaba-saba ve küfürlü ifadeler için sizden özür dilerim ancak modelimi test etmem için zorunluydu bu :) Umarım siz de benim kadar çok şey öğrenmişsinizdir. Yorumlarınız benim için çok kıymetli, lütfen yorum yapmaya çekinmeyin!